In [ ]:
!pip install keras_tuner

In [ ]:
# from google.colab import drive
import os
import re
import pickle
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from keras.preprocessing import text, sequence
from keras.layers import Input, Dense, Embedding, Flatten, Conv2D, MaxPool2D, Bidirectional, LSTM, GRU, concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D, SpatialDropout1D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.models import Model, Sequential
from keras import backend as K
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='loss', patience=10)#, verbose=1)
from sklearn.preprocessing import LabelEncoder
# HP
from kerastuner import Hyperband

from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score

# Data is unbalanced
import nltk
import string 

# keras Tuner
import keras_tuner as kt

In [ ]:
data_train = pd.read_excel('/kaggle/input/spam-review-detection/data_train.xlsx')
data_dev = pd.read_excel('/kaggle/input/spam-review-detection/data_dev.xlsx')
data_test = pd.read_excel('/kaggle/input/spam-review-detection/data_test.xlsx')

In [ ]:
y_train=data_train['Label']

y_dev=data_dev['Label']

y_test=data_test['Label']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=100)

In [ ]:
x_train_count=cv.fit_transform(data_train['transformed_text']).toarray()
x_dev_count=cv.transform(data_dev['transformed_text']).toarray()
x_test_count=cv.transform(data_test['transformed_text']).toarray()
x_train_count.shape

In [ ]:
from tensorflow.keras.layers import Dense, Input, Flatten, SimpleRNN, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [ ]:
x_train_rnn = x_train_count.reshape(14158, 1, 100)

In [ ]:
x_dev_count_RNN = x_dev_count.reshape(1580,1,100)

In [ ]:
x_test_count_RNN = x_test_count.reshape(3938,1,100)

In [ ]:
# model_RNN = tf.keras.Sequential()
def model_builder (hp):
  model_RNN = tf.keras.Sequential() 
  model_RNN.add(tf.keras.layers.Dense(
                  units = hp.Int('units0', min_value = 32, max_value = 512, step = 32), 
                  activation=hp.Choice('activation0', ['relu', 'tanh', 'sigmoid', 'softmax']),
                  input_shape = (1,100)))
  # Dropout
  model_RNN.add (tf.keras.layers.Dropout(hp.Float(f'dropout1', min_value = 0.1, max_value = 0.3, step = 0.05)))
    # SimpleRNN
  model_RNN.add(tf.keras.layers.SimpleRNN(
        units=hp.Int('units_rnn', min_value=32, max_value=512, step=32),
        activation=hp.Choice('activation_rnn', ['relu', 'tanh', 'sigmoid', 'softmax']),
        return_sequences=True))
  # loop
  for i in range (hp.Int('num_layers', 1, 8)):
    model_RNN.add(tf.keras.layers.Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                                        activation=hp.Choice(f'activation_{i}', ['relu', 'tanh', 'sigmoid', 'softmax'])                                        
                                       ))

    model_RNN.add(tf.keras.layers.Dropout(hp.Float(f'dropout_{i}', min_value=0.1, max_value=0.3, step=0.05)))

# Batch size 
  batch_size = hp.Int ('batch_size', 16, 128, step=32)

  model_RNN.add(tf.keras.layers.Dense(units = 1))

# Learning rate
  lr = hp.Float ('lr', 1e-4, 1e-2)
# Epochs
  num_epochs = hp.Int('num_epochs', min_value=5, max_value=20, step=5)
# Compile
  model_RNN.compile(
      loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
      optimizer=Adam(learning_rate=lr),
      metrics=['accuracy']
  )
  return model_RNN

In [ ]:
tuner_rnn = kt.Hyperband(
    model_builder,
    objective='val_accuracy',
    max_epochs = 10,
    factor = 3,
    directory='my_dir',
    project_name='my_project'
)
!rm -rf /content/tuner_dir_1

In [ ]:
hp = kt.HyperParameters()
tuner_rnn.search (x_train_rnn, 
              y_train, 
              epochs = 10,               
              validation_data = (x_dev_count_RNN, y_dev), 
              batch_size=hp.Int('batch_size', 16, 128, step=16), 
#               activation= activation,
              callbacks = [early_stop])

In [ ]:
best_hps_RNN = tuner_rnn.get_best_hyperparameters(num_trials=1)[0]
print(best_hps_RNN.values)

In [ ]:
print('Number of hidden layers:', best_hps_RNN.get('num_layers'))

for i in range(best_hps_RNN.get('num_layers')):
    print(f'Units for layer {i+1}:', best_hps_RNN.get(f'units_{i}'))
    print(f'Dropout for layer {i+1}:', best_hps_RNN.get(f'dropout_{i}'))
    if 'activation0' in best_hps_RNN:
        activation_fn = best_hps_RNN.get(f'activation_{i}')
        print(f'Activation Function for layer {i+1}:', activation_fn)
    else:
        print('Activation parameter does not exist.')
        
# print(best_hps.get("activation0"))
print('Learning rate:', best_hps_RNN.get('lr'))
print('Batch size:', best_hps_RNN.get('batch_size'))
print('Number of epochs:', best_hps_RNN.get('num_epochs'))
# print('Best Activavtion Function:', best_hps.get('activation{i}'))

In [ ]:
best_model = tuner_rnn.get_best_models ()[0]
best_model.build (x_train_rnn.shape)
best_model.summary()

In [ ]:
best_Model = tuner_rnn.hypermodel.build (best_hps_RNN)


In [ ]:
best_Model.fit(x_train_rnn, 
                y_train, 
                epochs = 20, 
                validation_data = (x_dev_count_RNN, y_dev), 
                batch_size=16
                )

In [ ]:
loss, accuracy = best_Model.evaluate(x_test_count_RNN, y_test)
